In [ ]:
import glob
import numpy as np
import preprocessing as pp
import matplotlib.pyplot as plt
from keras import backend as K
from IPython.display import Image as IM
from keras import layers, models, optimizers
from keras.utils.np_utils import to_categorical
from IPython.display import display_jpeg,display_png
from keras.applications import VGG16, Xception,ResNet50
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def original_model(in_shpae):
    model = models.Sequential()
    model.add(layers.Conv2D(
        4, (3, 3), activation = 'relu',padding = 'same',input_shape = in_shpae))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(16, (3, 3), activation= 'relu', padding= 'same'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv2D(128, (3,3), activation='relu', padding = 'same'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(256, (3,3), activation='relu', padding = 'same'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv2D(512, (3,3), activation='relu', padding = 'same'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(7, activation = 'softmax'))
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy'])
    
    return model

In [ ]:
def xception_model(in_shpae):
    return model

In [2]:
def vgg16_model():
    base_vgg = VGG16(
                    weights = 'imagenet',
                    include_top = False,
                    input_shape = X_train[0].shape)
    
    model = models.Sequential()
    model.add(base_vgg)
    model.add(layers.Conv2D(512, (3,3), activation='relu', padding = 'same'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(7, activation = 'softmax'))
    for layer in base_vgg.layers[:-4]:
        layer.trainable = False
    model.compile(optimizer=optimizers.RMSprop(lr = 1e-5),
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy'])
    return model

In [ ]:
# def train_generator():
#     train_datagen = ImageDataGenerator(rescale = 1./255)
#     return train_generator

In [3]:
# 前処理
X_train = []
y_train = []
datas = glob.glob('../data/*')
imsize = (224,224)
for data in datas:
    class_label = pp.data_to_label(data)
    img = pp.load_image(data,imsize)
    X = np.array(img)
    X_train.append(X)
    y_train.append(class_label)
X_train,y_train = pp.list_to_array(X_train, y_train)
y_train = to_categorical(y_train)

In [ ]:
# build_modelフェーズ
original_model = original_model(X_train[0].shape)
original_model.fit(X_train, y_train, epochs = 12)

In [ ]:
# xception_model = xception_model(in_shape)

In [4]:
vgg16_model = vgg16_model()
vgg16_model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
100/100 [==============================] - 32s 322ms/step - loss: 2.2408 - accuracy: 0.1000
Epoch 2/10
100/100 [==============================] - 35s 353ms/step - loss: 1.5266 - accuracy: 0.4800
Epoch 3/10
100/100 [==============================] - 37s 366ms/step - loss: 1.3118 - accuracy: 0.5700
Epoch 4/10
100/100 [==============================] - 36s 364ms/step - loss: 1.1683 - accuracy: 0.5800
Epoch 5/10
100/100 [==============================] - 37s 369ms/step - loss: 1.0668 - accuracy: 0.7500
Epoch 6/10
100/100 [==============================] - 36s 361ms/step - loss: 0.9797 - accuracy: 0.7600
Epoch 7/10
100/100 [==============================] - 33s 334ms/step - loss: 0.9091 - accuracy: 0.7700
Epoch 8/10
100/100 [==============================] - 34s 342ms/step - loss: 0.8310 - accuracy: 0.7800
Epoch 9/10
100/100 [==============================] - 35s 351ms/step - loss: 0.7614 - accuracy: 0.8400
Epoch 10/10
100/100 [==============================] - 33s 334ms/step - l

In [6]:
json_string = vgg16_model.to_json()
open('vgg16_model.json', 'w').write(json_string)
vgg16_model.save_weights('vgg16_model.hdf5')

In [ ]:
# 予測
X_test = []
test_data = './data/12_0052.png'
imsize = (224,224)
test_img = pp.load_image(test_data,imsize)
x_test = np.array(test_img)
X_test.append(x_test)
X_test = np.array(X_test)
# y_pred = original_model.predict(X_test)
y_pred = vgg16_model.predict(X_test)


In [ ]:
print('予想年齢:{}'.format(y_pred))
if test_data[-3:] =='png':
    display_png(IM(test_data))
elif test_data[-3:] =='jpg':
    display_jpeg(IM(test_data))

In [ ]:
vgg16_model.summary()

In [ ]:
# ヒートマップ
np.argmax(y_pred[0])
img_output = vgg16_model.output[:,1]
last_conv_layer = vgg16_model.get_layer('conv2d_31')
grads = K.gradients(img_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis=(0,1,2))
iterate = K.function([vgg16_model.input],[pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([X_test])
for i in range(512):
    conv_layer_output_value[: , : , i] *= pooled_grads_value[i]
hetamap = np.mean(conv_layer_output_value, axis=-1)

In [ ]:
heatmap = np.maximum(hetamap, 0)
heatmap/= np.max(heatmap)
plt.matshow(heatmap)

In [ ]:
import cv2
img3 = cv2.imread(test_data)
heatmap = cv2.resize(heatmap, (img3.shape[1], img3.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
saveimg = heatmap * 0.4 + img3
savename = './heat.jpg'
cv2.imwrite(savename, saveimg)
display_jpeg(IM(savename))

In [ ]:
#　labelの可視化
plt.hist(x = y_train,bins=7)